# backup

In [1]:
import os
import time
import subprocess
import gzip
import shutil

# --- Configuration ---
# --- IMPORTANT: Fill in your details below ---

# 1. Database Credentials
DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "1234"  # <-- IMPORTANT: REPLACE WITH YOUR PASSWORD
DB_NAME = "computer_shop"             # The database you want to back up. Use '--all-databases' for all.

# 2. Backup Directory
# The folder where you want to store your backups.
# Use double backslashes (\\) or a raw string (r'C:\path\to\backups').
BACKUP_DIR = "G:\CADT\CADT_Year_Two\DatabaseAdmin\computer_store\Backup"

# 3. Path to mysqldump.exe
# This is usually inside the 'bin' folder of your MySQL Server installation.
# Find the path and paste it here.
MYSQLDUMP_PATH = r"C:\Program Files\MySQL\MySQL Server 8.0\bin\mysqldump.exe"

# --- End of Configuration ---


def main():
    """
    Main function to perform the database backup.
    """
    print("Starting MySQL database backup...")
    print(f"Database: {DB_NAME}")
    print(f"Backup Directory: {BACKUP_DIR}")

    try:
        # Step 1: Create the backup directory if it doesn't exist
        if not os.path.exists(BACKUP_DIR):
            print(f"Creating backup directory: {BACKUP_DIR}")
            os.makedirs(BACKUP_DIR)

        # Step 2: Create a unique, timestamped filename for the backup
        timestamp = time.strftime('%Y-%m-%d_%H-%M-%S')
        sql_filename = f"{DB_NAME}_{timestamp}.sql"
        gzip_filename = f"{sql_filename}.gz"
        sql_filepath = os.path.join(BACKUP_DIR, sql_filename)
        gzip_filepath = os.path.join(BACKUP_DIR, gzip_filename)
        
        # Step 3: Construct the mysqldump command
        # Using a list of arguments is safer than a single command string.
        # Note: The password is included directly. See security note in the comments.
        dump_command = [
            MYSQLDUMP_PATH,
            f'--host={DB_HOST}',
            f'--user={DB_USER}',
            f'--password={DB_PASSWORD}',
            '--routines',         # Include stored procedures and functions
            '--triggers',         # Include triggers
            '--single-transaction', # Ensures a consistent state for InnoDB tables
            DB_NAME
        ]

        # Step 4: Execute the mysqldump command
        print(f"Running mysqldump for '{DB_NAME}'...")
        
        # We will pipe the output directly to a file
        with open(sql_filepath, 'wb') as f_out:
            process = subprocess.Popen(dump_command, stdout=f_out, stderr=subprocess.PIPE)
            
            # Wait for the process to complete and capture any errors
            stdout, stderr = process.communicate()

            if process.returncode != 0:
                # An error occurred
                error_message = stderr.decode('utf-8').strip()
                raise Exception(f"mysqldump failed with error: {error_message}")

        print(f"Successfully created SQL dump: {sql_filename}")

        # Step 5: Compress the SQL file using gzip
        print(f"Compressing backup file to: {gzip_filename}...")
        with open(sql_filepath, 'rb') as f_in:
            with gzip.open(gzip_filepath, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        
        # Step 6: Remove the uncompressed SQL file
        os.remove(sql_filepath)
        print("Compression successful. Original SQL file removed.")
        
        print("\n--- Backup Complete ---")
        print(f"Your compressed backup is located at: {gzip_filepath}")
        
    except FileNotFoundError:
        print("\n--- ERROR ---")
        print(f"Could not find 'mysqldump.exe' at the specified path: {MYSQLDUMP_PATH}")
        print("Please verify the path in the script's configuration section is correct.")
    except Exception as e:
        print(f"\n--- An unexpected error occurred ---")
        print(e)
        # Clean up the partial SQL file if an error occurred during compression
        if 'sql_filepath' in locals() and os.path.exists(sql_filepath):
            os.remove(sql_filepath)
            
if __name__ == "__main__":
    main()


<>:19: SyntaxWarning: invalid escape sequence '\C'
<>:19: SyntaxWarning: invalid escape sequence '\C'
C:\Users\vatha\AppData\Local\Temp\ipykernel_32\1277996885.py:19: SyntaxWarning: invalid escape sequence '\C'
  BACKUP_DIR = "G:\CADT\CADT_Year_Two\DatabaseAdmin\computer_store\Backup"


Starting MySQL database backup...
Database: computer_shop
Backup Directory: G:\CADT\CADT_Year_Two\DatabaseAdmin\computer_store\Backup
Running mysqldump for 'computer_shop'...
Successfully created SQL dump: computer_shop_2025-06-30_01-33-46.sql
Compressing backup file to: computer_shop_2025-06-30_01-33-46.sql.gz...
Compression successful. Original SQL file removed.

--- Backup Complete ---
Your compressed backup is located at: G:\CADT\CADT_Year_Two\DatabaseAdmin\computer_store\Backup\computer_shop_2025-06-30_01-33-46.sql.gz


# Restore

In [ ]:
import os
import sys
import subprocess
import gzip
import shutil

# --- Configuration ---
# --- IMPORTANT: Fill in your details below ---

# 1. Database Credentials
DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "Panhaygo7$"  # <-- IMPORTANT: REPLACE WITH YOUR PASSWORD
DB_NAME = "computer_shop"             # The database you want to restore.

# 2. Path to the Backup File to Restore
# This should be the full path to your compressed backup file (e.g., 'computer_shop_2025-06-30_01-26-00.sql.gz').
# Use double backslashes (\\) or a raw string.
BACKUP_FILE_PATH = r"D:\computer_shop_2025-06-30_01-33-46.sql.gz"

# 3. Path to mysql.exe
# This is usually inside the 'bin' folder of your MySQL Server installation.
# Find the path and paste it here.
MYSQL_CLIENT_PATH = r"C:\Program Files\MySQL\MySQL Server 8.0\bin\mysql.exe"

# --- End of Configuration ---


def main():
    """
    Main function to perform the database restore.
    """
    print("--- MySQL Database Restore Script ---")
    print(f"Database to restore: {DB_NAME}")
    print(f"Backup file: {BACKUP_FILE_PATH}")
    print("\n" + "="*40)
    print("!!! WARNING !!!")
    print(f"This operation will OVERWRITE all data in the '{DB_NAME}' database.")
    print("It is highly recommended to have a separate backup before proceeding.")
    print("="*40 + "\n")

    # Safety check: Confirm the user wants to proceed
    confirm = input("Are you sure you want to continue? (yes/no): ").lower()
    if confirm != 'yes':
        print("Restore operation cancelled by user.")
        sys.exit()

    # Define the path for the temporary decompressed SQL file
    sql_filepath = BACKUP_FILE_PATH.replace('.gz', '')

    try:
        # Step 1: Check for required files
        if not os.path.exists(MYSQL_CLIENT_PATH):
            raise FileNotFoundError(f"MySQL client not found at: {MYSQL_CLIENT_PATH}")
        if not os.path.exists(BACKUP_FILE_PATH):
            raise FileNotFoundError(f"Backup file not found at: {BACKUP_FILE_PATH}")

        # Step 2: Decompress the backup file
        print(f"\nDecompressing '{os.path.basename(BACKUP_FILE_PATH)}'...")
        with gzip.open(BACKUP_FILE_PATH, 'rb') as f_in:
            with open(sql_filepath, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print("Decompression successful.")

        # Step 3: Construct the mysql import command
        # The command will be executed by passing the .sql file to the standard input of mysql.exe
        import_command = [
            MYSQL_CLIENT_PATH,
            f'--host={DB_HOST}',
            f'--user={DB_USER}',
            f'--password={DB_PASSWORD}',
            DB_NAME
        ]

        # Step 4: Execute the mysql command to restore the database
        print(f"Restoring database '{DB_NAME}' from SQL file...")
        
        with open(sql_filepath, 'rb') as f_in:
            process = subprocess.Popen(import_command, stdin=f_in, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            
            # Wait for the process to complete and capture any errors
            stdout, stderr = process.communicate()

            if process.returncode != 0:
                # An error occurred during the import
                error_message = stderr.decode('utf-8').strip()
                raise Exception(f"MySQL import failed with error: {error_message}")

        print("\n--- Restore Complete ---")
        print(f"Database '{DB_NAME}' has been successfully restored.")

    except FileNotFoundError as e:
        print("\n--- ERROR ---")
        print(e)
        print("Please verify the file paths in the script's configuration section.")
    except Exception as e:
        print(f"\n--- An unexpected error occurred ---")
        print(e)
    finally:
        # Step 5: Clean up by removing the temporary decompressed SQL file
        if os.path.exists(sql_filepath):
            os.remove(sql_filepath)
            print("Temporary SQL file has been removed.")

if __name__ == "__main__":
    main()


--- MySQL Database Restore Script ---
Database to restore: computer_shop
Backup file: D:\computer_shop_2025-06-30_01-33-46.sql.gz

!!! WARNING !!!
This operation will OVERWRITE all data in the 'computer_shop' database.
It is highly recommended to have a separate backup before proceeding.

